In [26]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import warnings
warnings.filterwarnings(action='once')

!pip install sociophysicsdatahandler
from sociophysicsDataHandler import SociophysicsDataHandler

!cat auth.txt

dh = SociophysicsDataHandler()
dh.list_files('ut')

def fetchData(path):
  #Fetch and combine all data from a directory
  file_list = dh.list_files(path)
  n = file_list.name.size
  df_array = np.empty(n, dtype=object)

  for i in range(n):
    dh.fetch_prorail_data_from_path(path + "/" + file_list.name[i]);
    #Convert the date time column to actual date time
    dh.df['date_time_utc'] = pd.to_datetime(dh.df['date_time_utc'], unit = 'ms')
    #Swap x and y
    column_titles = {'date_time_utc':'date_time_utc', 'tracked_object':'tracked_object', 'x_pos':'y_pos', 'y_pos':'x_pos'}
    dh.df.rename(columns = column_titles, inplace = True)

    df_array[i] = dh.df

  return pd.concat(df_array)


file_path = 'ehv/platform2.1/20210330/'


#fLst = dh.list_files(file_path)
#fLst

def checkBounds(df, limits, x_tag = 'x_pos', y_tag = 'y_pos'):
  x_lower = limits[0]
  x_upper = limits[1]
  y_lower = limits[2]
  y_upper = limits[3]

  within_xbounds_series = (df[x_tag] < x_upper) & (df[x_tag] > x_lower)
  within_ybounds_series = (df[y_tag] < y_upper) & (df[y_tag] > y_lower)
  within_bounds_series = within_xbounds_series & within_ybounds_series
  return within_bounds_series

# fetch the data from the first file
df = fetchData(file_path);
df.head()

df_grouped_by_id = df.groupby('tracked_object')
df_object_start_location = df_grouped_by_id.first()
df_object_end_location = df_grouped_by_id.last()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
guest
YDPGL-XREXC-QGMZS-UAKEStargeting path /storage/surfsara/ProRail_USE_LL_data/ut
Files listed. Accessible as <this-object>.filelist
targeting path /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20210330/
Files listed. Accessible as <this-object>.filelist
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20210330//EHV_Platform2.1_2021033008_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20210330//EHV_Platform2.1_2021033009_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20210330//EHV_Platform2.1_2021033010_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20210330//EHV_Platform2.1_2021033011_trajecto

In [27]:
#Limits of the boarding area on the upper side (1) and lower side (2) of the platform in Eindhoven
boarding_limits_E_O = [-5e3, 5e3, -5e2, 1.5e3] #x_lower, x_upper, y_lower, y_upper
boarding_limits_STAIRS = [-5e3, 5e3, -4.5e3, -5e2]
boarding_limits_E_I = [-5e3, 5e3, -6.5e3, -4.5e3]
boarding_limits_1_1 = [5e3, 1e4, -2.5e3, 1e4] 
boarding_limits_1_2 = [5e3, 1e4, -1.2e4, -2.5e3]
boarding_limits_2_1 = [1e4, 2.8e4, -2.5e3, 1e4]
boarding_limits_2_2 = [1e4, 2.8e4, -1.2e4, -2.5e3]
boarding_limits_3_1 = [2.8e4, 5e4, -2.5e3, 1e4]
boarding_limits_3_2 = [2.8e4, 5e4, -1.2e4, -2.5e3]
boarding_limits_4_1 = [5e4, 7.5e4, -3e3, 1e4]
boarding_limits_4_2 = [5e4, 7.5e4, -1.2e4, -3e3]

In [28]:
df['E_O'] = checkBounds(df, boarding_limits_E_O).astype(float)
df['STAIRS'] = checkBounds(df, boarding_limits_STAIRS).astype(float)
df['E_I'] = checkBounds(df, boarding_limits_E_I).astype(float)
df['1_1'] = checkBounds(df, boarding_limits_1_1).astype(float)
df['1_2'] = checkBounds(df, boarding_limits_1_2).astype(float)
df['2_1'] = checkBounds(df, boarding_limits_2_1).astype(float)
df['2_2'] = checkBounds(df, boarding_limits_2_2).astype(float)
df['3_1'] = checkBounds(df, boarding_limits_3_1).astype(float)
df['3_2'] = checkBounds(df, boarding_limits_3_2).astype(float)
df['4_1'] = checkBounds(df, boarding_limits_4_1).astype(float)
df['4_2'] = checkBounds(df, boarding_limits_4_2).astype(float)
df_totals = df.groupby('date_time_utc').sum().drop(columns=['tracked_object', 'y_pos', 'x_pos'])
df_totals.head()

def area_of_limits(limits):
  return abs((limits[0]-limits[1]*(limits[2]-limits[3])))

area_E_O = area_of_limits(boarding_limits_E_O)
area_STAIRS = area_of_limits(boarding_limits_STAIRS)
area_E_I = area_of_limits(boarding_limits_E_I)
area_1_1 = area_of_limits(boarding_limits_1_1)
area_1_2 = area_of_limits(boarding_limits_1_2)
area_2_1 = area_of_limits(boarding_limits_2_1)
area_2_2 = area_of_limits(boarding_limits_2_2)
area_3_1 = area_of_limits(boarding_limits_3_1)
area_3_2 = area_of_limits(boarding_limits_3_2)
area_4_1 = area_of_limits(boarding_limits_4_1)
area_4_2 = area_of_limits(boarding_limits_4_2)

df_totals['density_E_O'] =  df_totals['E_O'] / area_E_O
df_totals['density_STAIRS'] =  df_totals['STAIRS'] / area_STAIRS
df_totals['density_E_I'] =  df_totals['E_I'] / area_E_I
df_totals['density_1_1'] =  df_totals['1_1'] / area_1_1
df_totals['density_1_2'] =  df_totals['1_2'] / area_1_2
df_totals['density_2_1'] =  df_totals['2_1'] / area_2_1
df_totals['density_2_2'] =  df_totals['2_2'] / area_2_2
df_totals['density_3_1'] =  df_totals['3_1'] / area_3_1
df_totals['density_3_2'] =  df_totals['3_2'] / area_3_2
df_totals['density_4_1'] =  df_totals['4_1'] / area_4_1
df_totals['density_4_2'] =  df_totals['4_2'] / area_4_2

df_totals.head()

#df_totals should contain the number of people in any area at any time and densities too
#density has unit person/ px^2

,E_O,STAIRS,E_I,1_1,1_2,2_1,2_2,3_1,3_2,4_1,...,density_STAIRS,density_E_I,density_1_1,density_1_2,density_2_1,density_2_2,density_3_1,density_3_2,density_4_1,density_4_2
date_time_utc,,,,,,,,,,,,,,,,,,,,,
2021-03-30 08:24:46.500,4.0,1.0,0.0,2.0,1.0,6.0,5.0,4.0,5.0,1.0,...,5.001250e-08,0.0,1.599936e-08,1.052576e-08,1.714237e-08,1.879629e-08,6.399713e-09,1.052570e-08,1.025588e-09,7.406859e-09
2021-03-30 08:24:46.700,4.0,1.0,0.0,2.0,1.0,7.0,4.0,4.0,5.0,1.0,...,5.001250e-08,0.0,1.599936e-08,1.052576e-08,1.999943e-08,1.503703e-08,6.399713e-09,1.052570e-08,1.025588e-09,8.888231e-09
2021-03-30 08:24:46.800,3.0,1.0,0.0,2.0,1.0,7.0,4.0,4.0,5.0,1.0,...,5.001250e-08,0.0,1.599936e-08,1.052576e-08,1.999943e-08,1.503703e-08,6.399713e-09,1.052570e-08,1.025588e-09,8.888231e-09
2021-03-30 08:24:46.900,3.0,1.0,0.0,2.0,1.0,7.0,4.0,4.0,5.0,1.0,...,5.001250e-08,0.0,1.599936e-08,1.052576e-08,1.999943e-08,1.503703e-08,6.399713e-09,1.052570e-08,1.025588e-09,8.888231e-09
2021-03-30 08:24:47.000,3.0,1.0,0.0,2.0,0.0,7.0,5.0,4.0,5.0,1.0,...,5.001250e-08,0.0,1.599936e-08,0.000000e+00,1.999943e-08,1.879629e-08,6.399713e-09,1.052570e-08,1.025588e-09,8.888231e-09
